In [ ]:
# create artificial dataset

import pandas as pd

data = {'measurementTimestamp': [1000000000000,
                                 1000000000001,
                                 1000000000002,
                                 1000000000003,
                                 1000000000004],
       'sourceMac': ['2d1ff804-c499-4163-b90f-003df1e4ec65',
                     '85a84e08-c967-41d0-b56b-522574796776',
                     '2c019c2d-7e9e-42b1-b3bd-1797c918d4a7',
                     '2c019c2d-7e9e-42b1-b3bd-1797c918d4a8',
                     'e1deda99-163f-4b68-9ade-e1f05d070bf2'],
       'coordinates': [[20,20],
                       [-20,40],
                       [-50,50],
                       [60,-20],
                       [40,30]],
       'x_error': [10,10,10,10,10],
       'y_error': [10,10,10,10,10]}
        
frame = pd.DataFrame(data)

In [ ]:
# create artificial dataset

import pandas as pd

data = {'measurementTimestamp': [1000000000001],
       'sourceMac': ['2d1ff804-c499-4163-b90f-003df1e4ec65'],
       'coordinates': [[20,20]],
       'x_error': [20],
       'y_error': [20]}
        
frame = pd.DataFrame(data)

In [ ]:
# real dataset

import json

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [ ]:
def randomMac():
   return str(uuid.uuid4())

In [ ]:
# create artificial dataset

import pandas as pd
import numpy as np
import uuid

data = {'sourceMac': pd.Series([randomMac() for i in range(1000)])}
frame = pd.DataFrame(data)

timestamps = np.arange(1000000000000,1000000001000,1)
coordinates = pd.Series([[0,0] for i in range(1000)])
x_error = pd.Series(np.ones(1000)) * 3.65
y_error = pd.Series(np.ones(1000)) * 3.65
frame['measurementTimestamp'] = timestamps
frame['coordinates'] = coordinates
frame['x_error'] = x_error
frame['y_error'] = y_error

In [ ]:
len(frame)

In [ ]:
data = {'sourceMac': pd.Series([mac for mac in set(frame['sourceMac'])])}

for i in range(1,10):
    
    df = pd.DataFrame(data)
    
    timestamps = np.arange(1000000000000 + i * 120000,\
                           1000000000000 + i * 120000 + 1000,1)
    coordinates = []
    for k in range(1000):
        x = frame['coordinates'].values[(i-1)*1000+k][0]
        y = frame['coordinates'].values[(i-1)*1000+k][1]
        for m in range(120):
            r1 = np.random.randn()
            r2 = np.random.randn()
            x += r1
            y += r2
        coordinates.append([x,y])
    coordinates = pd.Series(coordinates)
    x_error = pd.Series(np.zeros(1000))
    y_error = pd.Series(np.zeros(1000))
    df['measurementTimestamp'] = timestamps
    df['coordinates'] = coordinates
    df['x_error'] = x_error
    df['y_error'] = y_error
    
    dfs = [frame, df]
    
    frame = pd.concat(dfs)

In [ ]:
len(frame)

In [ ]:
# real dataset

import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [ ]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
chi2PerDof.columns = ['chi2PerDof']
probChi2 = pd.DataFrame(list(trackee['probChi2']))
probChi2.columns = ['probChi2']
nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
nMeasurements.columns = ['nMeasurements']

In [ ]:
# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
frame = frame.join(chi2PerDof)
frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(nMeasurements)
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [ ]:
frame = frame.sort_values(by='measurementTimestamp')

In [ ]:
frame[:3]

In [ ]:
def movingWindow(k):
    start = min(frame['measurementTimestamp']) + k * timestep
    stop = start + interval

    window = frame[(frame['measurementTimestamp'] >= start) & 
                       (frame['measurementTimestamp'] < stop)]

    return window

In [ ]:
def createHistograms(window):
    grids = np.zeros((len(set(window['sourceMac'])), height,width))

    # dictionary of histograms (with mac addresses as keys)
    histos = dict(zip(set(window['sourceMac']), grids))
    histos_old = dict(zip(set(window['sourceMac']), grids))
    
    # dictionary of normalization counts 
    norm_counts = dict( zip(set(window['sourceMac']), np.zeros(len(set(window['sourceMac'])))))
    
    return histos, norm_counts, histos_old

In [ ]:
def resetHistograms(histos):
    
    #print(histos_old['2d1ff804-c499-4163-b90f-003df1e4ec65'].sum())
    #print(histos['2d1ff804-c499-4163-b90f-003df1e4ec65'].sum())
    histos_old = copy.deepcopy(histos)
    #print(histos_old['2d1ff804-c499-4163-b90f-003df1e4ec65'].sum())
    
    grids = np.zeros((len(histos), height,width))
    histos = dict(zip(histos.keys(), grids))
    
    # dictionary of normalization counts 
    norm_counts = dict( zip(histos.keys(), np.zeros(len(histos))))
    
    return histos, norm_counts, histos_old

In [ ]:
def kernel(x, sigma): 
    return 1/(sqrt(2*pi) * sigma) * exp(-(x**2)/(2*sigma**2))

In [ ]:
def smoothHistogram(window, i, xbin, ybin):
    #### kernel density estimation #########################################

    sigma_x = window['x_error'].values[i] / 3
    sigma_y = window['y_error'].values[i] / 3
    
    smooth_bins = np.zeros((height, width))
    
    if sigma_x > 0.4 or sigma_y > 0.4:
        for u in range(width):
            for v in range(height):
                smooth_bins[v][u] += kernel((u - xbin), sigma_x) \
                * kernel((v - ybin), sigma_y)
    else:
        smooth_bins[ybin][xbin] += 1
    
    '''
    if sigma_x > 0.4 and sigma_y > 0.4:
        var = multivariate_normal(mean=[xbin,ybin], cov=[[sigma_x**2,0],[0,sigma_y**2]])
        for u in range(width):
                for v in range(height):
                    smooth_bins[v][u] += var.pdf([u,v])
    
    '''
    #if i > 30 and i < 60:
    #    print(sigma_x, sigma_y, smooth_bins.sum())

    ######################################################################
    
    return smooth_bins

In [ ]:
def updateHistograms(window, histos, norm_counts):
    
    for i in range(len(window)):
        # bin positions
        xbin = width/2  + floor(window['coordinates'].values[i][0] / 3)
        ybin = height/2 + floor(window['coordinates'].values[i][1] / 3)

        if xbin >= 0 and xbin < width and ybin >= 0 and ybin < height:
            
            smooth_bins = smoothHistogram(window, i, xbin, ybin)
            
            # update density histograms
            if not window['sourceMac'].values[i] in histos:
                histos[window['sourceMac'].values[i]] = np.zeros((height,width))
                norm_counts[window['sourceMac'].values[i]] = 0
                
            histos[window['sourceMac'].values[i]] += smooth_bins
            norm_counts[window['sourceMac'].values[i]] += 1
            
    return histos, norm_counts

In [ ]:
def normalizeHistograms(histos, histos_old, norm_counts):
    # total density histogram per period
    total_dens_histo = np.zeros((height, width))
    
    # generate weighting function with dispersion set to 
    # Brownian motion with v = 1 m/s and t = 120 sec
    
    var = multivariate_normal(mean=[39,29], cov=[[13.33,0],[0,13.33]])
    
    weights = np.zeros((60,80))
    for i in np.arange(80):
        for j in np.arange(60):
            weights[j][i] += var.pdf([i,j])
    
    for mac in histos.keys():
        if norm_counts[mac] == 0:
            # smooth existing pdf from previous time interval
            # apply a convolution
                    
            conv = signal.convolve2d(histos_old[mac], weights, boundary='wrap', mode='same')
            
            histos[mac] = conv
            
            total_dens_histo += conv
        else:
            total_dens_histo += histos[mac] / norm_counts[mac]
                
    return total_dens_histo, histos

In [ ]:
def runDataAnalysis():
    
    for k in range(periods):
        window = movingWindow(k)
        
        if k < 1:
            histos, norm_counts, histos_old = createHistograms(window)   
        else:
            histos, norm_counts, histos_old = resetHistograms(histos)
           
        histos, norm_counts = updateHistograms(window, histos, norm_counts)
        
        total_dens_histo, histos = normalizeHistograms(histos, histos_old, norm_counts)
        
        np.savetxt('output/test-com-histo_%d.csv' %  k, total_dens_histo, delimiter=',')
        print('Time window:', k)      

In [ ]:
from math import floor, ceil, sqrt, pi, exp
import numpy as np
from scipy.stats import multivariate_normal
from scipy import signal
import copy

# size of binned region (number of bins 3x3m)
width = 80; height = 60
# numbers of time intervals
periods = 30
timestep = 30000
interval = 120000

runDataAnalysis()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(16,10))

#col = ['r', 'y', 'c', 'k', 'c','r'] * height * width
col = ['w','r','w','w','w','w'] * height * width
# colors = np.random.choice(col, height*width)

for k in range(15,periods):
    
    ax = fig.add_subplot(111, projection='3d')

    x_data, y_data = np.meshgrid( np.arange(width),
                                  np.arange(height)*(-1) )

    x_data = x_data.flatten()
    y_data = y_data.flatten()

    z_data = np.loadtxt('output/test-com-histo_%d.csv' % k, delimiter=',').flatten()
    #z_data = total_dens_histos[k].flatten()
    ax.set_zlim3d(0, 18)
    ax.bar3d( x_data,
              y_data,
              np.zeros(len(z_data)),
              1, 1, z_data, color=col) # 
    if k < 10:
        number = '000' + str(k)
    elif k > 9:
        number = '00' + str(k)
    elif k > 99:
        number = '0' + str(k)
    plt.savefig('output/com-histograms-%s.png' % number)

#plt.show()